# TensorFlow style

In [1]:
import numpy as np

In [2]:
# train data and label
train = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

label = np.array([
    [0],
    [1],
    [1],
    [0]
])

In [3]:
# define a model
import tensorflow as tf

class MLP(tf.keras.Model):
    """
    多層パーセプトロン
    
    Attributes:
        l1 (tf.keras.layers.Dense): 1st layer (hidden layer)
        l2 (tf.keras.layers.Dense): 2nd layer (output layer)
    """

    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
    ):
        """
        constructor
        
        Args:
            input_dim (int): input dimension
            hidden_dim (int): hidden layer dimension
            output_dim (int): output layer dimension
        """
        super(MLP, self).__init__()
        self.l1 = tf.keras.layers.Dense(
            units=hidden_dim,
            input_dim=input_dim,
            activation='sigmoid',
        )
        self.l2 = tf.keras.layers.Dense(
            units=output_dim,
            activation='sigmoid'
        )

    @tf.function
    def call(
        self,
        x: np.ndarray,
        training: bool=False,
    ):
        """
        forward propagation
        
        Args:
            x (np.ndarray(float32)): input data
            training (bool): training flag
        Returns:
            np.ndarray(float32): output data
        """
        h = self.l1(x)
        y = self.l2(h)
        return y

In [4]:
# create a model
# input_dim=2, hidden_dim=2, output_dim=1
model = MLP(2, 2, 1)

/Users/takako/Desktop/GitHub/obj_recog_book/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# loss function and optimizer
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.5)

In [10]:
# training
@tf.function
def train_step(
    x: np.ndarray,
    t: np.ndarray,
) -> float:
    """
    training step
    
    Args:
        x (np.ndarray(float32)): input data
        t (np.ndarray(float32)): label data
    Returns:
        float: loss value
    """
    with tf.GradientTape() as tape:
        predictions = model(
            x,
            training=True # threre should be a training flag=True
        )
        loss = loss_fn(t, predictions)
    gradients = tape.gradient(
        loss,
        model.trainable_variables
    )
    optimizer.apply_gradients(
        zip(gradients, model.trainable_variables)
    )
    return loss

In [11]:
# training loop
epochs = 4000

for epoch in range(epochs):
    epoch_loss  = 0.0
    loss = train_step(train, label)
    epoch_loss += loss.numpy()
    if epoch % 1000 == 0:
        print(
            f'epoch: {epoch+1}: loss: {epoch_loss:.4f}'
        )

epoch: 1: loss: 0.7539
epoch: 1001: loss: 0.0414
epoch: 2001: loss: 0.0135
epoch: 3001: loss: 0.0079


In [12]:
model.summary()

Model: "mlp"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
# evaluate model prediction
print(model(train))

tf.Tensor(
[[0.00653716]
 [0.99497473]
 [0.9950243 ]
 [0.00568647]], shape=(4, 1), dtype=float32)


In [14]:
print(tf.cast(
    tf.math.greater(model(train), 0.5),
    tf.float32
))

tf.Tensor(
[[0.]
 [1.]
 [1.]
 [0.]], shape=(4, 1), dtype=float32)


# Keras style